In [85]:
import matplotlib.pyplot as plt

In [6]:
import random
from typing import Dict, Tuple

import numpy as np
from scipy import ndimage
from functools import partial
# from check_submission import check_submission
from game_mechanics import (
    Connect4Env,
    choose_move_randomly,
    get_empty_board,
    get_piece_longest_line_length,
    get_top_piece_row_index,
    has_won,
    is_column_full,
    load_dictionary,
    place_piece,
    play_connect_4_game,
    save_dictionary,
)
from tqdm import tqdm 

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from einops.layers.torch import Rearrange, Reduce
from einops import reduce, rearrange
from torchsummary import summary




TEAM_NAME = "WIP"  # <---- Enter your team name here!
assert TEAM_NAME != "Team Name", "Please change your TEAM_NAME!"

In [17]:
device = 'cpu'

In [73]:
network = nn.Sequential(
    nn.Conv2d(1,20, kernel_size=3, padding=True),
    nn.ReLU(),
    nn.Dropout2d(p=0.1),
    nn.BatchNorm2d(20),
    nn.Conv2d(1,20, kernel_size=3, padding=True),
    nn.ReLU(),
    nn.Dropout2d(p=0.1),
    nn.BatchNorm2d(20),
    nn.Conv2d(1,20, kernel_size=3, padding=True),
    nn.ReLU(),
    nn.Dropout2d(p=0.1),
    nn.BatchNorm2d(20),
    Rearrange('b w h c -> b (w h) c')
).to('cpu')

In [76]:
device = 'cpu'

In [77]:
torch_board2 = torch.randn(5,224,224, device=device)

In [81]:
score = {'won': 0, 'total': 0}
score['won'] += 1

In [69]:
numpy_board = np.random.randint(-1,2,(1,6,8,1))
torch_board = torch.Tensor(numpy_board)#.unsqueeze(0)#, device=device)#.detach()

In [88]:
network.forward(torch_board)

TypeError: conv2d() received an invalid combination of arguments - got (Tensor, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([32;1mTensor[0m, [31;1mParameter[0m, [31;1mParameter[0m, [31;1mtuple[0m, [31;1mtuple[0m, [31;1mtuple[0m, [32;1mint[0m)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([32;1mTensor[0m, [31;1mParameter[0m, [31;1mParameter[0m, [31;1mtuple[0m, [31;1mtuple[0m, [31;1mtuple[0m, [32;1mint[0m)


In [23]:
network(torch_board)

TypeError: conv2d() received an invalid combination of arguments - got (Tensor, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([32;1mTensor[0m, [31;1mParameter[0m, [31;1mParameter[0m, [31;1mtuple[0m, [31;1mtuple[0m, [31;1mtuple[0m, [32;1mint[0m)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([32;1mTensor[0m, [31;1mParameter[0m, [31;1mParameter[0m, [31;1mtuple[0m, [31;1mtuple[0m, [31;1mtuple[0m, [32;1mint[0m)


In [ ]:


def get_conv_block(units_in, units_out, dropout=0.2):
    return nn.Sequential(
        nn.Conv2d(units_in, units_out, kernel_size=3,padding=True,stride=2),
        nn.ReLU(),
        nn.Dropout2d(p=dropout),
        nn.BatchNorm2d(units_out))

layer_widths = [100,100,10]
layers = []
in_channels = 3
for out_channels in layer_widths:
    layers.append(get_conv_block(in_channels, out_channels))
    in_channels = out_channels
layers.append(Rearrange('b c w h -> b (c w h)'))
## divide by 4 for each layer, cause we have downsampling:
layers.append(nn.Linear(224**2*out_channels//(4**len(layer_widths)),10))
layers.append(nn.Softmax())
model = nn.Sequential(*layers).to(device)



In [ ]:
def create_network